# Tutorial

Welcome to this tutorial on using OpenFLASH! This guide demonstrates how to set up a multi-body hydrodynamic problem, run the simulation engine, and analyze the results.

OpenFLASH uses the **Matched Eigenfunction Expansion Method (MEEM)** to efficiently analyze wave interactions with concentric structures, providing key hydrodynamic insights like added mass, damping, and potential fields.

---

## 1. Prerequisites and Setup

Before you begin, make sure you have installed the `openflash` package. If you haven't, you can install it using `pip`. It's highly recommended to do this within a virtual environment.

```bash
# Install the package from your local project directory
pip install open-flash
```

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openflash
print(type(openflash))
print(openflash.__path__)
print(openflash.__file__)


# --- Import core modules from package ---
try:
    from openflash import *
    from openflash.multi_constants import g
    print("OpenFLASH modules imported successfully!")
except ImportError as e:
    print(f"Error importing OpenFLASH modules. Error: {e}")

# Set NumPy print options for better readability
np.set_printoptions(threshold=np.inf, linewidth=np.inf, precision=8, suppress=True)

<class 'module'>
['/Users/hopebest/Documents/semi-analytical-hydro/package/src/openflash']
/Users/hopebest/Documents/semi-analytical-hydro/package/src/openflash/__init__.py
OpenFLASH modules imported successfully!


In [ ]:
# ---------------------------------
# --- 1. Problem Setup ---
# ---------------------------------
print("\n--- 1. Setting up the Problem ---")
h = 1.001            # Water Depth (m)
d_list = [0.5, 0.25]   # Step depths (m) for inner and outer bodies
a_list = [0.5, 1.0]    # Radii (m) for inner and outer bodies
NMK = [30, 30, 30]     # Harmonics for inner, middle, and exterior domains

m0 = 1.0    # Non-dimensional wave number
problem_omega = omega(m0, h, g)
print(f"Wave number (m0): {m0}, Angular frequency (omega): {problem_omega:.4f}")

In [ ]:
def run_simulation(heaving_list, case_name):
    """Sets up geometry, problem, and engine for a specific heaving configuration."""
    print(f"\n--- Running Simulation: {case_name} ---")
    
    # 1. Create Bodies
    bodies = []
    for i in range(len(a_list)):
        body = SteppedBody(
            a=np.array([a_list[i]]),
            d=np.array([d_list[i]]),
            slant_angle=np.array([0.0]), 
            heaving=heaving_list[i]
        )
        bodies.append(body)

    # 2. Arrangement
    arrangement = ConcentricBodyGroup(bodies)

    # 3. Geometry
    geometry = BasicRegionGeometry(
        body_arrangement=arrangement,
        h=h,
        NMK=NMK
    )

    # 4. Problem
    problem = MEEMProblem(geometry)
    problem.set_frequencies(np.array([problem_omega]))
    
    # 5. Engine
    engine = MEEMEngine(problem_list=[problem])
    
    # 6. Solve
    print(f"Solving for {case_name}...")
    X = engine.solve_linear_system_multi(problem, m0)
    
    # 7. Coefficients
    coeffs = engine.compute_hydrodynamic_coefficients(problem, X, m0)
    
    return problem, X, coeffs


## 2. Case 1: Inner Body Heaving (Mode 0)

We simulate the case where only the inner body is heaving (`heaving_list = [True, False]`).

In [ ]:
problem1, X1, coeffs1 = run_simulation([True, False], "Inner Body Heaving")

print("\nHydrodynamic Coefficients (Mode 0):")
if coeffs1:
    print(pd.DataFrame(coeffs1))

## 3. Case 2: Outer Body Heaving (Mode 1)

Now we simulate the case where only the outer body is heaving (`heaving_list = [False, True]`).

**Note:** Currently, the package calculates the diagonal terms of the hydrodynamic coefficient matrix. Off-diagonal (coupling) terms are not yet implemented, so we run separate problems for each mode.

In [ ]:
problem2, X2, coeffs2 = run_simulation([False, True], "Outer Body Heaving")

print("\nHydrodynamic Coefficients (Mode 1):")
if coeffs2:
    print(pd.DataFrame(coeffs2))

## 4. Potential Field Visualization

We can visualize the potential field for one of the cases (e.g., Case 2: Outer Body Heaving).

In [ ]:
# Setup variables for visualization using Case 2 results
problem = problem2
X = X2


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openflash
print(type(openflash))
print(openflash.__path__)
print(openflash.__file__)


# --- Import core modules from package ---
try:
    from openflash import *
    from openflash.multi_constants import g
    print("OpenFLASH modules imported successfully!")
except ImportError as e:
    print(f"Error importing OpenFLASH modules. Error: {e}")

# Set NumPy print options for better readability
np.set_printoptions(threshold=np.inf, linewidth=np.inf, precision=8, suppress=True)